<a href="https://colab.research.google.com/github/ykitaguchi77/Colab_Scripts/blob/master/coreML%E3%82%92%E7%94%A8%E3%81%84%E3%81%A6MNIST%E3%82%92ios%E3%81%ABdeploy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**PytorchのスクリプトをCoreMLに変換**

参考サイト：<br>
https://qiita.com/shu223/items/6ddfbedb4fdfb2059a11
https://chemicalfactory.hatenablog.com/entry/2020/01/26/230114

In [2]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms

batch_size = 10

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transforms.ToTensor())
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                            shuffle=True, num_workers=0)

testset = torchvision.datasets.MNIST(root='./data', train=False, 
                                        download=True, transform=transforms.ToTensor())
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                            shuffle=False, num_workers=0)

class MNIST_Conv_MN(nn.Module):
    def __init__(self):
        super(MNIST_Conv_MN, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3) 
        self.pooling = nn.MaxPool2d(2, 2) 
        self.fc1 = nn.Linear(13 * 13 * 8, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pooling(x)
        x = x.view(-1, 13 * 13 * 8)
        x = self.fc1(x)
        return x
    
model=MNIST_Conv_MN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

num_epochs = 10

for epoch in range(num_epochs):
    
    train_loss = 0
    train_acc = 0
    val_loss = 0
    val_acc = 0
    
    model.train()
    for i, (inputs, labels) in enumerate(trainloader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        train_loss += loss.item()
        train_acc += (outputs.max(1)[1] == labels).sum().item()
        loss.backward()
        optimizer.step()
    avg_train_loss = train_loss / len(trainloader.dataset)
    avg_train_acc = train_acc / len(trainloader.dataset)

    model.eval()
    with torch.no_grad():
        for inputs, labels in testloader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            val_acc += (outputs.max(1)[1] == labels).sum().item()
        avg_val_loss = val_loss / len(testloader.dataset)
        avg_val_acc = val_acc / len(testloader.dataset)
        
        print("Epoch [{}/{}], Loss: {loss:.4f}, val_loss: {val_loss:.4f}, val_acc: {val_acc:.4f}"\
              .format(epoch+1, num_epochs, i+1, loss=avg_train_loss, val_loss=avg_val_loss, val_acc=avg_val_acc))
        
torch.save(model.state_dict(), "MNIST.pth")

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Epoch [1/10], Loss: 0.0250, val_loss: 0.0134, val_acc: 0.9623
Epoch [2/10], Loss: 0.0118, val_loss: 0.0099, val_acc: 0.9678
Epoch [3/10], Loss: 0.0085, val_loss: 0.0084, val_acc: 0.9738
Epoch [4/10], Loss: 0.0070, val_loss: 0.0068, val_acc: 0.9783
Epoch [5/10], Loss: 0.0061, val_loss: 0.0072, val_acc: 0.9774
Epoch [6/10], Loss: 0.0054, val_loss: 0.0069, val_acc: 0.9793
Epoch [7/10], Loss: 0.0049, val_loss: 0.0065, val_acc: 0.9794
Epoch [8/10], Loss: 0.0045, val_loss: 0.0069, val_acc: 0.9795
Epoch [9/10], Loss: 0.0041, val_loss: 0.0075, val_acc: 0.9774
Epoch [10/10], Loss: 0.0038, val_loss: 0.0070, val_acc: 0.9792


In [3]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms

#import coreML ver4.0
!pip install -U coremltools
import coremltools


class MNIST_Conv_MN(nn.Module):
    def __init__(self):
        super(MNIST_Conv_MN, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3) 
        self.pooling = nn.MaxPool2d(2, 2) 
        self.fc1 = nn.Linear(13 * 13 * 8, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pooling(x)
        x = x.view(-1, 13 * 13 * 8)
        x = self.fc1(x)
        return x
    
model = MNIST_Conv_MN()
model.load_state_dict(torch.load('MNIST.pth'))
scripted_model = torch.jit.script(model)

#サイズの合ったランダムな入力行う
example_input = torch.rand(1,1,28,28)
traced = torch.jit.trace(model, example_input)

#traced.save("model.pt")
mlmodel = coremltools.convert(
    traced, 
    inputs=[coremltools.TensorType(name="input", shape=example_input.shape)]
)
mlmodel.save('MNIST.mlmodel')

     |████████████████████████████████| 3.4MB 27.3MB/s 
  Created wheel for attr: filename=attr-0.3.1-cp37-none-any.whl size=2458 sha256=be51f2b5afc8cb97aed094b3a86fa261bdafb4ecc93a6c8a7b42dd0231770d9f
  Stored in directory: /root/.cache/pip/wheels/f0/96/9b/1f8892a707d17095b5a6eab0275da9d39e68e03a26aee2e726
Successfully built attr


Translating MIL ==> MLModel Ops: 100%|██████████| 19/19 [00:00<00:00, 4594.25 ops/s]


#サンプル文字をダウンロード

In [7]:
import numpy as np
from PIL import Image
from tensorflow.keras.datasets import mnist
import os
from google.colab import files

"""
#google driveをcolabolatoryにマウント
from google.colab import drive
drive.mount('/content/drive')

sample_dir = "/content/drive/MyDrive/Deep_learning/MNIST_samples"
"""

sample_dir = "/content/MNIST_samples"
n_samples = 30

#ダウンロード用のフォルダを作成

os.makedirs(sample_dir, exist_ok=True)

(x_train, y_train), (x_test, y_test) = mnist.load_data()
# 画像データ
# print(x_train[0])
# ラベルの値
for i in range(n_samples):
  print(y_train[i])
  img = Image.fromarray(np.uint8(x_train[i]))
  img.save(sample_dir +'/' + str(i) +'.png')

5
0
4
1
9
2
1
3
1
4
3
5
3
6
1
7
2
8
6
9
4
0
9
1
1
2
4
3
2
7


#**動作確認**



In [46]:
from PIL import Image
import random
import glob
from torchvision import transforms


#ダウンロードしたフォルダからランダムに画像を選択
input_image = Image.open(random.choice(glob.glob('/content/MNIST_samples/*')))
display(input_image)

#前処理

preprocess = transforms.Compose([
   transforms.ToTensor()
   #normalizeなどあればここに記載
])

input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(1)

In [47]:
with torch.no_grad():
   output = model(input_batch)[0]
torch_predictions = output.argmax(0)

print(int(torch_predictions))

4
